In [1]:
import arcpy

proj_path = 'CURRENT'

aprx = arcpy.mp.ArcGISProject(proj_path)
m = [m for m in aprx.listMaps() if m.name=='korindo'][0]
m.name

'korindo'

In [2]:
# ## merging into one tif from exported GEE
# arcpy.management.MosaicToNewRaster(
#     input_rasters="planet_with_indices_median_202404_2025070000010240-0000005120.tif;planet_with_indices_median_202404_2025070000010240-0000000000.tif;planet_with_indices_median_202404_2025070000005120-0000010240.tif;planet_with_indices_median_202404_2025070000005120-0000005120.tif;planet_with_indices_median_202404_2025070000005120-0000000000.tif;planet_with_indices_median_202404_2025070000000000-0000010240.tif;planet_with_indices_median_202404_2025070000000000-0000005120.tif;planet_with_indices_median_202404_2025070000000000-0000000000.tif;planet_with_indices_median_202404_2025070000010240-0000010240.tif",
#     output_location=r"\\Mac\Home\Documents",
#     raster_dataset_name_with_extension="mosaic_planet_korindo_median_202404_202507.tif",
#     coordinate_system_for_the_raster='PROJCRS["WGS_1984_UTM_Zone_49S",BASEGEOGCRS["GCS_WGS_1984",DYNAMIC[FRAMEEPOCH[1990.5],MODEL["AM0-2"]],DATUM["D_WGS_1984",ELLIPSOID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],CS[ellipsoidal,2],AXIS["Latitude (lat)",north,ORDER[1]],AXIS["Longitude (lon)",east,ORDER[2]],ANGLEUNIT["Degree",0.0174532925199433]],CONVERSION["Transverse_Mercator",METHOD["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",10000000.0],PARAMETER["Central_Meridian",111.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0]],CS[Cartesian,2],AXIS["Easting (X)",east,ORDER[1]],AXIS["Northing (Y)",north,ORDER[2]],LENGTHUNIT["Meter",1.0],ID["EPSG","32749"]]',
#     pixel_type="16_BIT_UNSIGNED",
#     cellsize=3,
#     number_of_bands=41,
#     mosaic_method="LAST",
#     mosaic_colormap_mode="FIRST"
# )

In [3]:
# ###OBIA cluster arcgis pro
# with arcpy.EnvManager(scratchWorkspace=r"\\Mac\Home\Documents\ArcGIS\Projects\treeo_analysis"):
#     out_raster_dataset = arcpy.ia.SegmentMeanShift(
#         in_raster="mosaic_planet_korindo_median_202404_202507.tif",
#         spectral_detail=15.5,
#         spatial_detail=15,
#         min_segment_size=278,
#         band_indexes="12,9,10",
#         max_segment_size=-1
#     )
#     out_raster_dataset.save(r"\\Mac\Home\Documents\ArcGIS\Projects\treeo_analysis\mosaic_planet__SegmentMeanShd1.crf")

In [7]:
# ## export cluster to features (vector)
# arcpy.conversion.RasterToPolygon(
#     in_raster="Extract_mosa1",
#     out_polygon_features=r"\\Mac\Home\Documents\ArcGIS\Projects\treeo_analysis\treeo_analysis.gdb\RasterT_Extract1",
#     simplify="NO_SIMPLIFY",
#     raster_field="OBJECTID",
#     create_multipart_features="SINGLE_OUTER_PART",
#     max_vertices_per_feature=None
# )

In [8]:
# ## convert back to raster
# arcpy.conversion.PolygonToRaster(
#     in_features="RasterT_Extract1",
#     value_field="OBJECTID",
#     out_rasterdataset=r"\\Mac\Home\Documents\ArcGIS\Projects\treeo_analysis\treeo_analysis.gdb\RasterT_Extract1_PolygonToRaster",
#     cell_assignment="CELL_CENTER",
#     priority_field="NONE",
#     cellsize=3,
#     build_rat="DO_NOT_BUILD"
# )

In [9]:
# 1. Set your inputs
segment_raster = [l for l in m.listLayers() if l.name=="RasterT_Extract1_PolygonToRaster"][0].dataSource  # Drag and drop your cluster layer here
original_image = [l for l in m.listLayers() if l.name=="mosaic_planet_korindo_median_202404_202507.tif"][0].dataSource # Drag and drop your original image here
output_gdb = arcpy.env.workspace # Uses your project's default geodatabase
segment_raster

'\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\treeo_analysis\\treeo_analysis.gdb\\RasterT_Extract1_PolygonToRaster'

In [23]:
import arcpy
import os

# --- SETUP ---
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "100%"

# --- INPUTS ---
# Assume 'original_image' and 'segment_raster' are defined
desc = arcpy.Describe(original_image)
bands = desc.children 
zone_raster = segment_raster

# 1. Define stats as a list to loop through
stat_types = ["MEDIAN", "MAXIMUM", "MEAN", "MINIMUM", "STD"]

print(f"Found {len(bands)} bands in {desc.baseName}")

processed_rasters = [] # Stores file paths for the composite
band_names_map = []    # Stores the logical names (e.g., "B9_MED") for renaming later

final_output_name = "Final_Segmented_Composite"

print(f"Starting Processing...")

# --- OUTER LOOP: BANDS ---
for i, band in enumerate(bands, start=1):
    
    # Filter for Band 9 and 10 only
    if i >= 9:
        print(f"--- Processing Band {i}: {band.name} ---")
        
        # Sanitize Name
        clean_band_name = band.name.replace(".", "_").replace("-", "_")
        in_band_path = os.path.join(desc.catalogPath, band.name)

        # --- INNER LOOP: STATISTICS ---
        # We must run Zonal Stats once for EACH statistic type
        for stat in stat_types:
            
            # Create a unique name: e.g., "Recon_Band_9_MEDIAN"
            out_raster_name = f"Recon_{clean_band_name}_{stat}"
            
            # print(f"   > Calculating {stat}...")

            # 2. Run Zonal Statistics (In Memory)
            out_ras_obj = arcpy.sa.ZonalStatistics(
                in_zone_data=zone_raster,
                zone_field="Value",
                in_value_raster=in_band_path,
                statistics_type=stat, # PASS SINGLE STRING HERE
                ignore_nodata="DATA"
            )
            
            # 3. ROBUST SAVE (Using CopyRaster)
            full_out_path = os.path.join(arcpy.env.workspace, out_raster_name)
            
            if arcpy.Exists(full_out_path):
                arcpy.management.Delete(full_out_path)

            try:
                # Save the individual stat raster
                arcpy.management.CopyRaster(
                    in_raster=out_ras_obj,
                    out_rasterdataset=out_raster_name,
                    pixel_type="32_BIT_FLOAT" 
                )
                
                # Add to list for compositing
                processed_rasters.append(full_out_path)
                
                # Store the intended short name for the final band
                # e.g., "B9_MEDIAN" (Keep it short for GDB)
                short_name = f"B{i}_{stat}"
                band_names_map.append(short_name)
                
                print(f"     > Saved: {out_raster_name}")
                
            except Exception as e:
                print(f"     > ERROR Saving {stat}: {e}")

# --- 4. COMPOSITE & RENAME BANDS ---
if processed_rasters:
    print("--- Creating Composite ---")
    
    # A. Create the Composite (Bands will be named Band_1, Band_2...)
    arcpy.management.CompositeBands(processed_rasters, final_output_name)
    
    print("--- Renaming Internal Bands ---")
    # B. The "Pro Trick" to rename bands inside the composite
    # We iterate through the bands of the NEW composite and rename them.
    
    composite_path = os.path.join(arcpy.env.workspace, final_output_name)
    
    try:
        # Loop through 1 to N bands
        for i in range(len(band_names_map)):
            band_index = i + 1
            new_name = band_names_map[i] # e.g., "B9_MEDIAN"
            
            # The path to a specific band in a GDB is "CompositeName/Band_1"
            old_band_path = os.path.join(composite_path, f"Band_{band_index}")
            
            # Rename it!
            arcpy.management.Rename(old_band_path, new_name)
            print(f"   > Renamed Band_{band_index} to {new_name}")
            
        print(f"SUCCESS! {final_output_name} created with correct band info.")
        
    except Exception as e:
        print(f"Composite created, but renaming bands failed (Common in GDBs): {e}")
        print("The bands are in this order:", band_names_map)

else:
    print("No rasters processed. Check your loop conditions.")

Found 41 bands in mosaic_planet_korindo_median_202404_202507
Starting Processing...
--- Processing Band 9: Band_9 ---
     > Saved: Recon_Band_9_MEDIAN
     > Saved: Recon_Band_9_MAXIMUM
     > Saved: Recon_Band_9_MEAN
     > Saved: Recon_Band_9_MINIMUM
     > Saved: Recon_Band_9_STD
--- Processing Band 10: Band_10 ---
     > Saved: Recon_Band_10_MEDIAN
     > Saved: Recon_Band_10_MAXIMUM
     > Saved: Recon_Band_10_MEAN
     > Saved: Recon_Band_10_MINIMUM
     > Saved: Recon_Band_10_STD
--- Processing Band 11: Band_11 ---
     > Saved: Recon_Band_11_MEDIAN
     > Saved: Recon_Band_11_MAXIMUM
     > Saved: Recon_Band_11_MEAN
     > Saved: Recon_Band_11_MINIMUM
     > Saved: Recon_Band_11_STD
--- Processing Band 12: Band_12 ---
     > Saved: Recon_Band_12_MEDIAN
     > Saved: Recon_Band_12_MAXIMUM
     > Saved: Recon_Band_12_MEAN
     > Saved: Recon_Band_12_MINIMUM
     > Saved: Recon_Band_12_STD
--- Processing Band 13: Band_13 ---
     > Saved: Recon_Band_13_MEDIAN
     > Saved: Recon_